In [9]:
import numpy as np
import pandas as pd

In [10]:
match = pd.read_csv('matches.csv')
delivery = pd.read_csv('deliveries.csv')

In [11]:
# To predict extract columns: Batting Team, Bowling Team, City, Runs Left, Balls left, Wickets Left, Total Runs, CRR, RRR, Result

In [12]:
match.head()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan


In [13]:
match.shape

(950, 20)

In [14]:
delivery.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [15]:
delivery.shape

(225954, 17)

In [16]:
total_score_df = delivery.groupby(['ID','innings','BattingTeam']).sum()['total_run'].reset_index()  # reset_index is used to convert it into dataframe

In [17]:
total_score_df

,ID,innings,BattingTeam,total_run
0,335982,1,Kolkata Knight Riders,222
1,335982,2,Royal Challengers Bangalore,82
2,335983,1,Chennai Super Kings,240
3,335983,2,Kings XI Punjab,207
4,335984,1,Rajasthan Royals,129
...,...,...,...,...
1923,1312198,2,Lucknow Super Giants,193
1924,1312199,1,Royal Challengers Bangalore,157
1925,1312199,2,Rajasthan Royals,161
1926,1312200,1,Rajasthan Royals,130


In [18]:
total_score_df = total_score_df[total_score_df['innings']==1]

In [19]:
total_score_df

,ID,innings,BattingTeam,total_run
0,335982,1,Kolkata Knight Riders,222
2,335983,1,Chennai Super Kings,240
4,335984,1,Rajasthan Royals,129
6,335985,1,Mumbai Indians,165
8,335986,1,Deccan Chargers,110
...,...,...,...,...
1918,1304116,1,Sunrisers Hyderabad,157
1920,1312197,1,Rajasthan Royals,188
1922,1312198,1,Royal Challengers Bangalore,207
1924,1312199,1,Royal Challengers Bangalore,157


In [20]:
match_df = match.merge(total_score_df[['ID','total_run']],left_on='ID', right_on='ID')

In [21]:
match_df.head()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,...,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,total_run
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,130
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,...,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon,157
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,...,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough,207
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,...,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma,188
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,...,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan,157


In [22]:
match_df['Team1'].unique()

array(['Rajasthan Royals', 'Royal Challengers Bangalore',
       'Sunrisers Hyderabad', 'Delhi Capitals', 'Chennai Super Kings',
       'Gujarat Titans', 'Lucknow Super Giants', 'Kolkata Knight Riders',
       'Punjab Kings', 'Mumbai Indians', 'Kings XI Punjab',
       'Delhi Daredevils', 'Rising Pune Supergiant', 'Gujarat Lions',
       'Rising Pune Supergiants', 'Pune Warriors', 'Deccan Chargers',
       'Kochi Tuskers Kerala'], dtype=object)

In [23]:
teams = [
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders',
    'Punjab Kings',
    'Chennai Super Kings',
    'Rajasthan Royals',
    'Delhi Capitals',
    'Gujarat Titans',
    'Lucknow Super Giants'
]

In [24]:
# Changing old names to new names
match_df['Team1'] = match_df['Team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['Team2'] = match_df['Team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['Team1'] = match_df['Team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['Team2'] = match_df['Team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

match_df['Team1'] = match_df['Team1'].str.replace('Kings XI Punjab','Punjab Kings')
match_df['Team2'] = match_df['Team2'].str.replace('Kings XI Punjab','Punjab Kings')

In [25]:
# Keeping rows of only those teams who plays IPL now. 
match_df = match_df[match_df['Team1'].isin(teams)]
match_df = match_df[match_df['Team2'].isin(teams)]

In [26]:
match_df.shape

(835, 21)

In [27]:
match_df.head()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,...,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,total_run
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,130
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,...,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon,157
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,...,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough,207
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,...,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma,188
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,...,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan,157


In [28]:
match_df['method'].value_counts()
# 15 matches rain effected.

method
D/L    15
Name: count, dtype: int64

In [29]:
# match_df = match_df[match_df['method'] == 'NaN']

In [30]:
# match_df['method'].value_counts()

In [31]:
match_df

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,...,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,total_run
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,130
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,...,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon,157
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,...,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough,207
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,...,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma,188
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,...,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,335986,Kolkata,2008-04-20,2007/08,4,Kolkata Knight Riders,Sunrisers Hyderabad,Eden Gardens,Deccan Chargers,bat,...,Kolkata Knight Riders,Wickets,5.0,NaN,DJ Hussey,"['WP Saha', 'BB McCullum', 'RT Ponting', 'SC G...","['AC Gilchrist', 'Y Venugopal Rao', 'VVS Laxma...",BF Bowden,K Hariharan,110
946,335985,Mumbai,2008-04-20,2007/08,5,Mumbai Indians,Royal Challengers Bangalore,Wankhede Stadium,Mumbai Indians,bat,...,Royal Challengers Bangalore,Wickets,5.0,NaN,MV Boucher,"['L Ronchi', 'ST Jayasuriya', 'DJ Thornely', '...","['S Chanderpaul', 'R Dravid', 'LRPL Taylor', '...",SJ Davis,DJ Harper,165
947,335984,Delhi,2008-04-19,2007/08,3,Delhi Capitals,Rajasthan Royals,Feroz Shah Kotla,Rajasthan Royals,bat,...,Delhi Daredevils,Wickets,9.0,NaN,MF Maharoof,"['G Gambhir', 'V Sehwag', 'S Dhawan', 'MK Tiwa...","['T Kohli', 'YK Pathan', 'SR Watson', 'M Kaif'...",Aleem Dar,GA Pratapkumar,129
948,335983,Chandigarh,2008-04-19,2007/08,2,Punjab Kings,Chennai Super Kings,"Punjab Cricket Association Stadium, Mohali",Chennai Super Kings,bat,...,Chennai Super Kings,Runs,33.0,NaN,MEK Hussey,"['K Goel', 'JR Hopes', 'KC Sangakkara', 'Yuvra...","['PA Patel', 'ML Hayden', 'MEK Hussey', 'MS Dh...",MR Benson,SL Shastri,240


In [32]:
match_df = match_df[['ID','City','WinningTeam','total_run']]

In [33]:
# joining with delivery data
delivery_df = match_df.merge(delivery, on='ID')

In [34]:
# to predict, we need to work on data of 2nd innings
delivery_df = delivery_df[delivery_df['innings']==2]

In [35]:
delivery_df.shape

(95974, 20)

In [36]:
delivery_df.head()

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
120,1312200,Ahmedabad,Gujarat Titans,130,2,0,1,WP Saha,TA Boult,Shubman Gill,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans
121,1312200,Ahmedabad,Gujarat Titans,130,2,0,2,WP Saha,TA Boult,Shubman Gill,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans
122,1312200,Ahmedabad,Gujarat Titans,130,2,0,3,WP Saha,TA Boult,Shubman Gill,NaN,1,0,1,0,0,NaN,NaN,NaN,Gujarat Titans
123,1312200,Ahmedabad,Gujarat Titans,130,2,0,4,Shubman Gill,TA Boult,WP Saha,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans
124,1312200,Ahmedabad,Gujarat Titans,130,2,0,5,Shubman Gill,TA Boult,WP Saha,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans


In [37]:
# creating new column as 'current_score'
delivery_df['current_score'] = delivery_df.groupby('ID')['total_run_y'].cumsum()

In [38]:
delivery_df.head()

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,...,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,current_score
120,1312200,Ahmedabad,Gujarat Titans,130,2,0,1,WP Saha,TA Boult,Shubman Gill,...,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,0
121,1312200,Ahmedabad,Gujarat Titans,130,2,0,2,WP Saha,TA Boult,Shubman Gill,...,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,0
122,1312200,Ahmedabad,Gujarat Titans,130,2,0,3,WP Saha,TA Boult,Shubman Gill,...,1,0,1,0,0,NaN,NaN,NaN,Gujarat Titans,1
123,1312200,Ahmedabad,Gujarat Titans,130,2,0,4,Shubman Gill,TA Boult,WP Saha,...,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,1
124,1312200,Ahmedabad,Gujarat Titans,130,2,0,5,Shubman Gill,TA Boult,WP Saha,...,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,1


In [39]:
delivery_df['runs_left'] = delivery_df['total_run_x'] - delivery_df['current_score'] + 1

In [40]:
delivery_df

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,...,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,current_score,runs_left
120,1312200,Ahmedabad,Gujarat Titans,130,2,0,1,WP Saha,TA Boult,Shubman Gill,...,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131
121,1312200,Ahmedabad,Gujarat Titans,130,2,0,2,WP Saha,TA Boult,Shubman Gill,...,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131
122,1312200,Ahmedabad,Gujarat Titans,130,2,0,3,WP Saha,TA Boult,Shubman Gill,...,0,1,0,0,NaN,NaN,NaN,Gujarat Titans,1,130
123,1312200,Ahmedabad,Gujarat Titans,130,2,0,4,Shubman Gill,TA Boult,WP Saha,...,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130
124,1312200,Ahmedabad,Gujarat Titans,130,2,0,5,Shubman Gill,TA Boult,WP Saha,...,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198930,335982,Bangalore,Kolkata Knight Riders,222,2,14,5,P Kumar,I Sharma,SB Joshi,...,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,80,143
198931,335982,Bangalore,Kolkata Knight Riders,222,2,14,6,SB Joshi,I Sharma,P Kumar,...,0,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,81,142
198932,335982,Bangalore,Kolkata Knight Riders,222,2,14,7,P Kumar,I Sharma,SB Joshi,...,0,0,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,81,142
198933,335982,Bangalore,Kolkata Knight Riders,222,2,15,1,SB Joshi,LR Shukla,P Kumar,...,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,82,141


In [41]:
delivery_df.head()

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,...,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,current_score,runs_left
120,1312200,Ahmedabad,Gujarat Titans,130,2,0,1,WP Saha,TA Boult,Shubman Gill,...,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131
121,1312200,Ahmedabad,Gujarat Titans,130,2,0,2,WP Saha,TA Boult,Shubman Gill,...,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131
122,1312200,Ahmedabad,Gujarat Titans,130,2,0,3,WP Saha,TA Boult,Shubman Gill,...,0,1,0,0,NaN,NaN,NaN,Gujarat Titans,1,130
123,1312200,Ahmedabad,Gujarat Titans,130,2,0,4,Shubman Gill,TA Boult,WP Saha,...,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130
124,1312200,Ahmedabad,Gujarat Titans,130,2,0,5,Shubman Gill,TA Boult,WP Saha,...,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130


In [42]:
delivery_df['balls_left'] = 120 - (delivery_df['overs']*6 + delivery_df['ballnumber'])

In [43]:
delivery_df.head()

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,...,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,current_score,runs_left,balls_left
120,1312200,Ahmedabad,Gujarat Titans,130,2,0,1,WP Saha,TA Boult,Shubman Gill,...,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131,119
121,1312200,Ahmedabad,Gujarat Titans,130,2,0,2,WP Saha,TA Boult,Shubman Gill,...,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131,118
122,1312200,Ahmedabad,Gujarat Titans,130,2,0,3,WP Saha,TA Boult,Shubman Gill,...,1,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,117
123,1312200,Ahmedabad,Gujarat Titans,130,2,0,4,Shubman Gill,TA Boult,WP Saha,...,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,116
124,1312200,Ahmedabad,Gujarat Titans,130,2,0,5,Shubman Gill,TA Boult,WP Saha,...,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,115


In [44]:
delivery_df

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,...,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,current_score,runs_left,balls_left
120,1312200,Ahmedabad,Gujarat Titans,130,2,0,1,WP Saha,TA Boult,Shubman Gill,...,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131,119
121,1312200,Ahmedabad,Gujarat Titans,130,2,0,2,WP Saha,TA Boult,Shubman Gill,...,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131,118
122,1312200,Ahmedabad,Gujarat Titans,130,2,0,3,WP Saha,TA Boult,Shubman Gill,...,1,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,117
123,1312200,Ahmedabad,Gujarat Titans,130,2,0,4,Shubman Gill,TA Boult,WP Saha,...,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,116
124,1312200,Ahmedabad,Gujarat Titans,130,2,0,5,Shubman Gill,TA Boult,WP Saha,...,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198930,335982,Bangalore,Kolkata Knight Riders,222,2,14,5,P Kumar,I Sharma,SB Joshi,...,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,80,143,31
198931,335982,Bangalore,Kolkata Knight Riders,222,2,14,6,SB Joshi,I Sharma,P Kumar,...,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,81,142,30
198932,335982,Bangalore,Kolkata Knight Riders,222,2,14,7,P Kumar,I Sharma,SB Joshi,...,0,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,81,142,29
198933,335982,Bangalore,Kolkata Knight Riders,222,2,15,1,SB Joshi,LR Shukla,P Kumar,...,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,82,141,29


In [45]:
delivery_df['player_out'] = delivery_df['player_out'].fillna("0")
delivery_df['player_out'] = delivery_df['player_out'].apply(lambda x:x if x == "0" else "1")
delivery_df['player_out'] = delivery_df['player_out'].astype('int')
wickets = delivery_df.groupby('ID')['player_out'].cumsum()
delivery_df['wickets_left'] = 10 - wickets
delivery_df.head()

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,...,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,current_score,runs_left,balls_left,wickets_left
120,1312200,Ahmedabad,Gujarat Titans,130,2,0,1,WP Saha,TA Boult,Shubman Gill,...,0,0,0,NaN,NaN,Gujarat Titans,0,131,119,10
121,1312200,Ahmedabad,Gujarat Titans,130,2,0,2,WP Saha,TA Boult,Shubman Gill,...,0,0,0,NaN,NaN,Gujarat Titans,0,131,118,10
122,1312200,Ahmedabad,Gujarat Titans,130,2,0,3,WP Saha,TA Boult,Shubman Gill,...,0,0,0,NaN,NaN,Gujarat Titans,1,130,117,10
123,1312200,Ahmedabad,Gujarat Titans,130,2,0,4,Shubman Gill,TA Boult,WP Saha,...,0,0,0,NaN,NaN,Gujarat Titans,1,130,116,10
124,1312200,Ahmedabad,Gujarat Titans,130,2,0,5,Shubman Gill,TA Boult,WP Saha,...,0,0,0,NaN,NaN,Gujarat Titans,1,130,115,10


In [46]:
delivery_df.tail()

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,...,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,current_score,runs_left,balls_left,wickets_left
198930,335982,Bangalore,Kolkata Knight Riders,222,2,14,5,P Kumar,I Sharma,SB Joshi,...,0,0,0,NaN,NaN,Royal Challengers Bangalore,80,143,31,1
198931,335982,Bangalore,Kolkata Knight Riders,222,2,14,6,SB Joshi,I Sharma,P Kumar,...,0,0,0,NaN,NaN,Royal Challengers Bangalore,81,142,30,1
198932,335982,Bangalore,Kolkata Knight Riders,222,2,14,7,P Kumar,I Sharma,SB Joshi,...,0,0,0,NaN,NaN,Royal Challengers Bangalore,81,142,29,1
198933,335982,Bangalore,Kolkata Knight Riders,222,2,15,1,SB Joshi,LR Shukla,P Kumar,...,0,0,0,NaN,NaN,Royal Challengers Bangalore,82,141,29,1
198934,335982,Bangalore,Kolkata Knight Riders,222,2,15,2,SB Joshi,LR Shukla,P Kumar,...,0,1,1,caught,BB McCullum,Royal Challengers Bangalore,82,141,28,0


In [47]:
delivery_df.head()

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,...,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,current_score,runs_left,balls_left,wickets_left
120,1312200,Ahmedabad,Gujarat Titans,130,2,0,1,WP Saha,TA Boult,Shubman Gill,...,0,0,0,NaN,NaN,Gujarat Titans,0,131,119,10
121,1312200,Ahmedabad,Gujarat Titans,130,2,0,2,WP Saha,TA Boult,Shubman Gill,...,0,0,0,NaN,NaN,Gujarat Titans,0,131,118,10
122,1312200,Ahmedabad,Gujarat Titans,130,2,0,3,WP Saha,TA Boult,Shubman Gill,...,0,0,0,NaN,NaN,Gujarat Titans,1,130,117,10
123,1312200,Ahmedabad,Gujarat Titans,130,2,0,4,Shubman Gill,TA Boult,WP Saha,...,0,0,0,NaN,NaN,Gujarat Titans,1,130,116,10
124,1312200,Ahmedabad,Gujarat Titans,130,2,0,5,Shubman Gill,TA Boult,WP Saha,...,0,0,0,NaN,NaN,Gujarat Titans,1,130,115,10


In [48]:
# crr = runs/(balls_played/6)
delivery_df['crr'] = (delivery_df['current_score']*6)/(120-delivery_df['balls_left'])

In [49]:
delivery_df.head()

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,...,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,current_score,runs_left,balls_left,wickets_left,crr
120,1312200,Ahmedabad,Gujarat Titans,130,2,0,1,WP Saha,TA Boult,Shubman Gill,...,0,0,NaN,NaN,Gujarat Titans,0,131,119,10,0.0
121,1312200,Ahmedabad,Gujarat Titans,130,2,0,2,WP Saha,TA Boult,Shubman Gill,...,0,0,NaN,NaN,Gujarat Titans,0,131,118,10,0.0
122,1312200,Ahmedabad,Gujarat Titans,130,2,0,3,WP Saha,TA Boult,Shubman Gill,...,0,0,NaN,NaN,Gujarat Titans,1,130,117,10,2.0
123,1312200,Ahmedabad,Gujarat Titans,130,2,0,4,Shubman Gill,TA Boult,WP Saha,...,0,0,NaN,NaN,Gujarat Titans,1,130,116,10,1.5
124,1312200,Ahmedabad,Gujarat Titans,130,2,0,5,Shubman Gill,TA Boult,WP Saha,...,0,0,NaN,NaN,Gujarat Titans,1,130,115,10,1.2


In [50]:
delivery_df['rrr'] = (delivery_df['runs_left']*6)/delivery_df['balls_left']

In [51]:
delivery_df.head()

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,...,player_out,kind,fielders_involved,BattingTeam,current_score,runs_left,balls_left,wickets_left,crr,rrr
120,1312200,Ahmedabad,Gujarat Titans,130,2,0,1,WP Saha,TA Boult,Shubman Gill,...,0,NaN,NaN,Gujarat Titans,0,131,119,10,0.0,6.605042
121,1312200,Ahmedabad,Gujarat Titans,130,2,0,2,WP Saha,TA Boult,Shubman Gill,...,0,NaN,NaN,Gujarat Titans,0,131,118,10,0.0,6.661017
122,1312200,Ahmedabad,Gujarat Titans,130,2,0,3,WP Saha,TA Boult,Shubman Gill,...,0,NaN,NaN,Gujarat Titans,1,130,117,10,2.0,6.666667
123,1312200,Ahmedabad,Gujarat Titans,130,2,0,4,Shubman Gill,TA Boult,WP Saha,...,0,NaN,NaN,Gujarat Titans,1,130,116,10,1.5,6.724138
124,1312200,Ahmedabad,Gujarat Titans,130,2,0,5,Shubman Gill,TA Boult,WP Saha,...,0,NaN,NaN,Gujarat Titans,1,130,115,10,1.2,6.782609


In [52]:
def result(row):
    return 1 if row['BattingTeam'] == row['WinningTeam'] else 0

In [53]:
delivery_df['result'] = delivery_df.apply(result,axis=1)

In [54]:
delivery_df.head()

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,...,kind,fielders_involved,BattingTeam,current_score,runs_left,balls_left,wickets_left,crr,rrr,result
120,1312200,Ahmedabad,Gujarat Titans,130,2,0,1,WP Saha,TA Boult,Shubman Gill,...,NaN,NaN,Gujarat Titans,0,131,119,10,0.0,6.605042,1
121,1312200,Ahmedabad,Gujarat Titans,130,2,0,2,WP Saha,TA Boult,Shubman Gill,...,NaN,NaN,Gujarat Titans,0,131,118,10,0.0,6.661017,1
122,1312200,Ahmedabad,Gujarat Titans,130,2,0,3,WP Saha,TA Boult,Shubman Gill,...,NaN,NaN,Gujarat Titans,1,130,117,10,2.0,6.666667,1
123,1312200,Ahmedabad,Gujarat Titans,130,2,0,4,Shubman Gill,TA Boult,WP Saha,...,NaN,NaN,Gujarat Titans,1,130,116,10,1.5,6.724138,1
124,1312200,Ahmedabad,Gujarat Titans,130,2,0,5,Shubman Gill,TA Boult,WP Saha,...,NaN,NaN,Gujarat Titans,1,130,115,10,1.2,6.782609,1


In [55]:
total_score_df = total_score_df.rename(columns={'BattingTeam': 'BowlingTeam'})
delivery_df = delivery_df.merge(total_score_df[['ID','BowlingTeam']],on='ID')

In [56]:
delivery_df.head()

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,...,fielders_involved,BattingTeam,current_score,runs_left,balls_left,wickets_left,crr,rrr,result,BowlingTeam
0,1312200,Ahmedabad,Gujarat Titans,130,2,0,1,WP Saha,TA Boult,Shubman Gill,...,NaN,Gujarat Titans,0,131,119,10,0.0,6.605042,1,Rajasthan Royals
1,1312200,Ahmedabad,Gujarat Titans,130,2,0,2,WP Saha,TA Boult,Shubman Gill,...,NaN,Gujarat Titans,0,131,118,10,0.0,6.661017,1,Rajasthan Royals
2,1312200,Ahmedabad,Gujarat Titans,130,2,0,3,WP Saha,TA Boult,Shubman Gill,...,NaN,Gujarat Titans,1,130,117,10,2.0,6.666667,1,Rajasthan Royals
3,1312200,Ahmedabad,Gujarat Titans,130,2,0,4,Shubman Gill,TA Boult,WP Saha,...,NaN,Gujarat Titans,1,130,116,10,1.5,6.724138,1,Rajasthan Royals
4,1312200,Ahmedabad,Gujarat Titans,130,2,0,5,Shubman Gill,TA Boult,WP Saha,...,NaN,Gujarat Titans,1,130,115,10,1.2,6.782609,1,Rajasthan Royals


In [57]:
final_df = delivery_df[['BattingTeam','BowlingTeam','City','runs_left','balls_left','wickets_left','total_run_x','crr','rrr','result']]

In [58]:
final_df

,BattingTeam,BowlingTeam,City,runs_left,balls_left,wickets_left,total_run_x,crr,rrr,result
0,Gujarat Titans,Rajasthan Royals,Ahmedabad,131,119,10,130,0.000000,6.605042,1
1,Gujarat Titans,Rajasthan Royals,Ahmedabad,131,118,10,130,0.000000,6.661017,1
2,Gujarat Titans,Rajasthan Royals,Ahmedabad,130,117,10,130,2.000000,6.666667,1
3,Gujarat Titans,Rajasthan Royals,Ahmedabad,130,116,10,130,1.500000,6.724138,1
4,Gujarat Titans,Rajasthan Royals,Ahmedabad,130,115,10,130,1.200000,6.782609,1
...,...,...,...,...,...,...,...,...,...,...
95969,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,143,31,1,222,5.393258,27.677419,0
95970,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,142,30,1,222,5.400000,28.400000,0
95971,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,142,29,1,222,5.340659,29.379310,0
95972,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,141,29,1,222,5.406593,29.172414,0


In [59]:
# shuffling the data to avoid bias prediction.
final_df = final_df.sample(final_df.shape[0])

In [60]:
final_df

,BattingTeam,BowlingTeam,City,runs_left,balls_left,wickets_left,total_run_x,crr,rrr,result
8997,Kolkata Knight Riders,Royal Challengers Bangalore,Sharjah,78,68,8,138,7.038462,6.882353,1
61498,Royal Challengers Bangalore,Rajasthan Royals,Bangalore,58,68,8,117,6.923077,5.117647,1
24739,Kings XI Punjab,Royal Challengers Bangalore,Bengaluru,90,45,7,202,9.040000,12.000000,0
93443,Delhi Daredevils,Mumbai Indians,Mumbai,33,16,3,162,7.500000,12.375000,0
86011,Kings XI Punjab,Chennai Super Kings,Centurion,121,75,8,185,8.666667,9.680000,0
...,...,...,...,...,...,...,...,...,...,...
10150,Kolkata Knight Riders,Sunrisers Hyderabad,Dubai,10,13,7,115,5.943925,4.615385,1
38096,Kolkata Knight Riders,Delhi Daredevils,Kolkata,0,21,7,160,9.757576,0.000000,1
95941,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,172,58,3,222,4.935484,17.793103,0
93505,Deccan Chargers,Royal Challengers Bangalore,Bangalore,102,68,8,156,6.346154,9.000000,0


In [61]:
final_df.sample()

,BattingTeam,BowlingTeam,City,runs_left,balls_left,wickets_left,total_run_x,crr,rrr,result
51572,Kolkata Knight Riders,Chennai Super Kings,Kolkata,119,96,10,154,9.0,7.4375,1


In [62]:
final_df.isnull().sum()

BattingTeam        0
BowlingTeam        0
City            6012
runs_left          0
balls_left         0
wickets_left       0
total_run_x        0
crr                0
rrr               11
result             0
dtype: int64

In [63]:
final_df.dropna(inplace=True) # droping null values

In [64]:
# removing balls left =0 because it will create infinity rrr.
final_df = final_df[final_df['balls_left'] != 0]  

In [65]:
from sklearn.model_selection import train_test_split
X = final_df.iloc[:,:-1]
y = final_df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [66]:
X_train

,BattingTeam,BowlingTeam,City,runs_left,balls_left,wickets_left,total_run_x,crr,rrr
91815,Deccan Chargers,Delhi Daredevils,Delhi,168,103,9,194,9.529412,9.786408
78917,Kings XI Punjab,Kolkata Knight Riders,Kolkata,106,64,9,200,10.178571,9.937500
85999,Kings XI Punjab,Chennai Super Kings,Centurion,135,86,9,185,9.000000,9.418605
15115,Kolkata Knight Riders,Mumbai Indians,Chennai,29,22,5,152,7.591837,7.909091
15312,Sunrisers Hyderabad,Kolkata Knight Riders,Chennai,129,73,8,187,7.531915,10.602740
...,...,...,...,...,...,...,...,...,...
72372,Rajasthan Royals,Chennai Super Kings,Jaipur,181,109,10,196,8.727273,9.963303
12402,Sunrisers Hyderabad,Rajasthan Royals,Delhi,102,38,6,220,8.707317,16.105263
65473,Mumbai Indians,Royal Challengers Bangalore,Bangalore,166,108,8,171,3.000000,9.222222
58620,Kolkata Knight Riders,Royal Challengers Bangalore,Ranchi,79,80,8,115,5.550000,5.925000


In [67]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse_output=False,drop='first'),['BattingTeam','BowlingTeam','City'])
]
,remainder = 'passthrough')                        

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [69]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2', LogisticRegression(solver='liblinear'))
])


# # Accuracy of random forest is very high but the probabilities of winning between two teams will be one-sided. That's why we will use
# # LogisticRegression because probabilities of both teams will be smooth

# pipe = Pipeline(steps=[
#     ('step1',trf),
#     ('step2', RandomForestClassifier())
# ])

In [70]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['BattingTeam', 'BowlingTeam',
                                                   'City'])])),
                ('step2', LogisticRegression(solver='liblinear'))])

In [71]:
X_train.describe()

,runs_left,balls_left,wickets_left,total_run_x,crr,rrr
count,71728.000000,71728.000000,71728.000000,71728.000000,71728.000000,71728.000000
mean,93.207478,63.112174,7.567338,164.900722,7.439454,10.451635
std,49.606694,33.252314,2.127690,28.912654,2.269538,13.898571
min,-15.000000,-2.000000,0.000000,62.000000,0.000000,-516.000000
25%,54.000000,35.000000,6.000000,147.000000,6.275862,7.216216
50%,93.000000,64.000000,8.000000,165.000000,7.500000,8.906250
75%,131.000000,92.000000,9.000000,185.000000,8.666667,10.941176
max,247.000000,119.000000,10.000000,246.000000,36.000000,588.000000


In [72]:
y_pred = pipe.predict(X_test)

In [73]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8160374728154799

In [74]:
pipe.predict_proba(X_test)[10]

array([0.50034095, 0.49965905])

In [75]:
def match_summary(row):
    print("Batting Team-" + row['BattingTeam'] + " | Bowling Team-" + row['BowlingTeam'] + " | Target- " + str(row['total_run_x']))

In [76]:
teams

['Sunrisers Hyderabad',
 'Mumbai Indians',
 'Royal Challengers Bangalore',
 'Kolkata Knight Riders',
 'Punjab Kings',
 'Chennai Super Kings',
 'Rajasthan Royals',
 'Delhi Capitals',
 'Gujarat Titans',
 'Lucknow Super Giants']

In [77]:
delivery_df['City'].unique()

array(['Ahmedabad', 'Kolkata', 'Mumbai', 'Navi Mumbai', 'Pune', 'Dubai',
       'Sharjah', 'Abu Dhabi', 'Delhi', 'Chennai', nan, 'Hyderabad',
       'Visakhapatnam', 'Chandigarh', 'Bengaluru', 'Jaipur', 'Indore',
       'Bangalore', 'Raipur', 'Ranchi', 'Cuttack', 'Dharamsala', 'Nagpur',
       'Johannesburg', 'Centurion', 'Durban', 'Bloemfontein',
       'Port Elizabeth', 'Kimberley', 'East London', 'Cape Town'],
      dtype=object)

In [78]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))  # wb: write binary mode